In [ ]:
# Naive Bayes Classification for SID

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Install necessary libraries
!pip install nltk scikit-learn

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from gensim import models
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load your dataset (replace 'your_dataset.csv' with your dataset file)
path = "/content/drive/MyDrive/Saddam_Sir/Thesis/Dataset/Merged-file-_1_.csv"
df = pd.read_csv(path )

# Preprocess text data
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00


In [4]:
def preprocess_text(text):
    # Tokenization
    words = nltk.word_tokenize(str(text).lower())

    # Remove stopwords and non-alphanumeric characters
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]

    # Stemming and Lemmatization
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]

    # Reconstruct the cleaned text
    cleaned_text = ' '.join(lemmatized_words)

    return cleaned_text

df['cleaned_text'] = df['text'].apply(preprocess_text)

In [5]:
df = df.replace([np.inf, -np.inf], np.nan).fillna(99999)
df.replace([np.inf, -np.inf], np.nan).dropna()

,text,sentiment,cleaned_text
0,b'@SmurphySuzanne @Sunderland_GM And more coun...,1,b smurphysuzann countri align russia china nex...
1,b'RT @TatasenkoMana: This Method suicide 18 Da...,1,tatasenkomana method suicid 18 dazai explain o...
2,b'RT @ennui365: Why in the name of Vince Foste...,1,ennui365 name vinc foster heard sinc august fo...
3,b'@Alan_G_Smith_BC @diana_murphy613 @JustinTru...,0,b justintrudeau one drive someon suicid
4,"b""RT @sidhant: Turkey's mega Antalya Diplomati...",0,b rt sidhant turkey mega antalya diplomat foru...
...,...,...,...
1235,I am tired of being weak and destroyed.,1,tire weak destroy
1236,"Every religion fights a final, worldwide cru...",1,everi religion fight final worldwid crusad dea...
1237,should i stop wanting to be happy?,1,stop want happi
1238,I will spend the coming 207 days combing thr...,0,spend come 207 day comb scientif articl compil...


In [6]:
# Split the data into features (X) and labels (y)
X = df['cleaned_text']
y = df['sentiment'] # Replace 'Class' with your target column name
y = y.tolist()

In [7]:
from sklearn.preprocessing import MinMaxScaler #fixed import
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
import numpy as np

# Assuming X and y are lists
X_train = np.array(X).reshape(-1, 1)
y_train = np.array(y).reshape(-1, 1)


In [9]:
# Convert elements in X_train to strings if they are not already
X_train = [str(text) for text in X_train]

# Tokenize the cleaned text for Word2Vec
tokenized_text = [nltk.word_tokenize(text) for text in X_train]



In [10]:
# Train Word2Vec embeddings (you can adjust the parameters)
word2vec_model = models.Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, sg=0)

In [11]:
# Create Word2Vec vectors for the training data
X_train_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in text if word in word2vec_model.wv], axis=0) for text in tokenized_text])

In [12]:
# Tokenize and create Word2Vec vectors for the test data
tokenized_test = [nltk.word_tokenize(text) for text in X_test]
X_test_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in text if word in word2vec_model.wv], axis=0) for text in tokenized_test])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-12-f3696ed85135>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in text if word in word2vec_model.wv], axis=0) for text in tokenized_test])


In [13]:
# Assuming X_train_word2vec and X_test_word2vec are lists
X_train_word2vec = np.array(X_train_word2vec)
X_test_word2vec = np.array(X_test_word2vec)

In [14]:
# Check the shapes of your arrays
print(X_train_word2vec.shape)
print(X_test_word2vec.shape)

(1240, 100)
(248,)


In [15]:
import numpy as np

# Define the embedding dimension based on your Word2Vec model
embedding_dim = word2vec_model.vector_size

# Average Word2Vec embeddings for each document in the training and test sets
X_train_word2vec_avg = []
for doc in tokenized_text:
    embeddings = [word2vec_model.wv[word] for word in doc if word in word2vec_model.wv]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
    else:
        avg_embedding = np.zeros(embedding_dim)  # Use zeros for documents with no valid words
    X_train_word2vec_avg.append(avg_embedding)

X_train_word2vec_avg = np.array(X_train_word2vec_avg)

# Tokenize and create Word2Vec vectors for the test data
tokenized_test = [nltk.word_tokenize(text) for text in X_test]
X_test_word2vec_avg = []
for doc in tokenized_test:
    embeddings = [word2vec_model.wv[word] for word in doc if word in word2vec_model.wv]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
    else:
        avg_embedding = np.zeros(embedding_dim)  # Use zeros for documents with no valid words
    X_test_word2vec_avg.append(avg_embedding)

X_test_word2vec_avg = np.array(X_test_word2vec_avg)


In [16]:
# Import Logistic Regression
from sklearn.linear_model import LogisticRegression

# Create and train a Logistic Regression classifier
logistic_classifier = LogisticRegression(max_iter=1000, random_state=42)
logistic_classifier.fit(X_train_word2vec_avg, y_train)

# Predict on the test data
y_pred = logistic_classifier.predict(X_test_word2vec_avg)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation results
print("Test Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", report)


Test Accuracy: 0.6532258064516129

Confusion Matrix:
 [[  0  12  47]
 [  0  22  27]
 [  0   0 140]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.65      0.45      0.53        49
       99999       0.65      1.00      0.79       140

    accuracy                           0.65       248
   macro avg       0.43      0.48      0.44       248
weighted avg       0.50      0.65      0.55       248



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precisio